\setcounter{secnumdepth}{2}
\setcounter{tocdepth}{2}


# Abstract

## Resumen (ES)

**Palabras clave:** cinemática directa e inversa, DH estándar, jacobiano, singularidades, LSPB, tpoly, trayectoria cartesiana, ScanArm, MATLAB. ([GitHub][1])


Este proyecto presenta el modelado, análisis y simulación de un brazo articulado tipo **ScanArm** de **6 grados de libertad** parametrizado mediante **Denavit–Hartenberg estándar** (unidades: **m**). A partir de la tabla DH propuesta, se desarrolla la **cinemática directa** para reconstruir la pose del efector y la **cinemática inversa** para planificar movimientos hacia poses objetivo. Se estudia la **relación velocidad-articular** mediante el **jacobiano**, identificando **singularidades** y zonas de bajo condicionamiento relevantes para tareas de medición. En planificación, se implementan trayectorias **P2P** con perfiles **LSPB** y transiciones **quintic (tpoly)**, y trayectorias **cartesianas en SE(3)** para barridos de escaneo. El trabajo contempla dos escenarios: **uso pasivo** (medición guiada por operario, cinemática directa con encoders) y una **variante cooperativa actuada** para evaluación académica, donde la CI habilita la ejecución automática de vías en un entorno controlado. Se reportan simulaciones del **workspace**, perfiles cinemáticos y animaciones del recorrido, así como criterios de seguridad y límites articulares. El desarrollo se integra en **MATLAB** dentro del repositorio del curso (`Simulation - Final/`), con scripts reproducibles y figuras exportables para el informe. ([GitHub Repository][1])

---

## Abstract (EN)

This project presents the modeling, analysis, and simulation of a **ScanArm-type** **6-DoF** articulated manipulator parameterized with **standard Denavit–Hartenberg** notation (units in **meters**). Based on the proposed DH table, we derive **forward kinematics** to reconstruct the end-effector pose and **inverse kinematics** to plan motions toward target poses. We analyze the **joint-to-task velocity mapping** through the **Jacobian**, highlighting **singularities** and poor-conditioning regions that matter for measurement tasks. For motion planning, we implement **point-to-point** profiles using **LSPB** and **quintic (tpoly)** timing, and **Cartesian SE(3) paths** tailored to scanning sweeps. Two use cases are considered: a **passive** operation (human-guided measurement, forward kinematics via encoders) and a **cooperative actuated** variant for academic evaluation, where IK enables automatic execution of puntos de referencia under safe limits. We report simulations of the **workspace**, joint/Cartesian profiles, and rendered animations, together with safety criteria and joint bounds. The implementation is provided in **MATLAB** within the course repository (`Simulation - Final/`), including reproducible scripts and exportable figures for the final report. ([GitHGitHub Repositoryub][2])

**Keywords:** forward and inverse kinematics, standard DH, Jacobian, singularities, LSPB, tpoly, Cartesian trajectory, ScanArm, MATLAB. ([GitHub][1])

[1]: https://github.com/jerovidela/RoboticaI "GitHub - jerovidela/RoboticaI: Repositorio para trabajo grupal. Se puede usar como un google drive pero con control de versiones."
[2]: https://github.com/jerovidela/RoboticaI/tree/main/tp8 "RoboticaI/tp8 at main · jerovidela/RoboticaI · GitHub"


\pagebreak

# Introducción

La inspección geométrica asistida por brazos articulados de medición ha ganado relevancia en entornos donde se requiere movilidad, alcance y rapidez de despliegue. En este proyecto se modela y analiza un manipulador tipo ScanArm de **6 grados de libertad**, parametrizado mediante la convención **Denavit–Hartenberg estándar** con unidades en **metros**. El objetivo general es establecer una base cinemática sólida para tareas de escaneo y palpado, evaluando el desempeño geométrico del sistema y su capacidad para generar trayectorias que preserven la calidad metrológica.

El trabajo aborda tanto la **cinemática directa** como la **cinemática inversa** del modelo propuesto, junto con el **jacobiano** y el estudio de **singularidades** y condicionamiento. Si bien muchos dispositivos comerciales operan en modo **pasivo** (movidos por el operario y sensados por encoders), aquí se incorpora una **variante cooperativa actuada** con fines académicos: la CI se utiliza para planificar y, de ser pertinente, ejecutar vías cartesianas y articulares bajo límites de posición, velocidad y aceleración.

Para la **planificación de movimiento**, se emplean perfiles **LSPB** y polinomios **quintic** para movimientos punto a punto, y trayectorias **cartesianas en SE(3)** adecuadas a barridos de escaneo. Se analizan criterios de seguridad, límites articulares y regiones mal condicionadas del jacobiano, evitando configuraciones cercanas a singularidades. Las simulaciones incluyen estimación del **espacio de trabajo**, perfiles cinemáticos y animaciones de recorridos representativos.

El desarrollo se implementa en **MATLAB** y se integra en el repositorio del curso (`RoboticaI/Simulation - Final`), con scripts reproducibles y exportables para documentar resultados. 

Este documento se organiza de la siguiente manera: primero se presenta la parametrización DH y la validación de la cinemática directa; luego la cinemática inversa y el análisis de velocidades; a continuación la planificación de trayectorias en espacio articular y cartesiano; se muestran las simulaciones y resultados; y finalmente se discuten conclusiones y líneas de trabajo futuro.


# Tareas a resolver y robot seleccionado

## Problema a resolver

La célula requiere medición y escaneo dimensional con agilidad: piezas cambiantes, sin utillajes específicos, sin una CMM fija y sin depender cada semana de la agenda y el presupuesto de un tercero. El objetivo es disponer de un medio con alcance volumétrico medio sobre mesa, capacidad de orientar la normal del efector para barridos, repetibilidad suficiente para práctica académica e inspección ligera, y un flujo reproducible que permita simular y documentar resultados de forma consistente.

## Robot seleccionado y pertinencia

Se adopta un brazo articulado tipo ScanArm de seis grados de libertad, modelado con la convención Denavit–Hartenberg (**DH**) estándar en metros. Esta arquitectura en serie ofrece un equilibrio favorable entre alcance y compacidad de la celda; los seis ejes permiten posicionar el TCP (**Tool Center Point**) y gobernar su orientación para seguir superficies, condición necesaria en barridos de escaneo. En operación pasiva, los encoders de las juntas y el modelo DH proporcionan la cinemática directa para reconstruir la pose; en la variante cooperativa actuada, la cinemática inversa habilita la ejecución de vías planificadas y el análisis de límites, singularidades y perfiles de movimiento. 

La planificación articular y cartesiana con trayectoria con perfil de velocidad trapezoidal (**LSPB**) y polinomios de quinto orden (**quintic**) evita sacudidas y respeta límites de posición, velocidad y aceleración, preservando la calidad de la medición.


![image.png](Pictures/robot3.png)


## Conveniencia frente a alternativas

Frente a la contratación de un metrólogo externo, esta solución reduce la dependencia de agenda y el coste recurrente, a la vez que internaliza el conocimiento y garantiza la reproducibilidad mediante scripts y simulación previa. 

Una CMM puente superaría la precisión, pero su coste, huella espacial y falta de portabilidad la vuelven desproporcionada para una célula didáctica o procesos ágiles. 

Los instrumentos manuales (calibres, plantillas) son económicos y veloces, aunque poco versátiles y sin capacidad para capturar geometría 3D completa. 

Un robot industrial estándar aporta potencia y automatización, pero exige integración de seguridad y suele ser excesivo en costo y espacio para inspección flexible. 

El 6R tipo ScanArm, en cambio, es portátil, versátil, orientable y totalmente guionable, además de alinearse con los criterios de la cátedra (DH, CD/CI, jacobiano, trayectorias y simulación). Si se desea avanzar a control cooperativo, la transición es natural sin modificar el modelo geométrico.

Resumiendo las caracteristicas de cada uno:



## Alcance verificable del proyecto

El trabajo demostrará, con base en el repositorio, la definición y justificación de la tabla DH; la cinemática directa validada en posturas de prueba; la cinemática inversa numérica para planificar poses objetivo; el jacobiano con análisis de manipulabilidad y singularidades aplicadas al escaneo; la planificación de trayectorias articulares y cartesianas con temporización LSPB y quintic; y simulaciones del espacio de trabajo, perfiles cinemáticos y animaciones. Se incorporan criterios de seguridad y límites de juntas en todos los casos.

## Nota sobre las demostraciones

Las validaciones se realizarán mediante trayectorias simples y reproducibles: movimientos punto a punto entre referencias, un arco manteniendo la normal del TCP y un raster lineal sobre un plano virtual. En modo pasivo, estas trayectorias sirven como guía para el operario; en modo cooperativo actuado, se ejecutan con cinemática inversa respetando los límites definidos.

# Información del Scan Arm

El Scan Arm considerado es un manipulador 6R concebido para **medición y escaneo** guiados por el usuario (modo pasivo) y, en nuestra variante académica, también para **planificación/ejecución asistida** (modo cooperativo). Su mérito no es mover cargas paletizadas ni soldar chasis; es **posicionar** con libertad la punta y **orientarla** con precisión sobre superficies, requiriendo un costo financiero y espacial menor al de una CMM.

## Concepto y principio de operación

El brazo trabaja como **AACMM** (*Articulated Arm Coordinate Measuring Machine* o *Máquina de Medición por Coordenadas de brazo articulado*): una cadena cinemática en serie con **encoders** y **motores** en cada junta. En modo pasivo, el operario desplaza el brazo y el sistema reconstruye la pose del TCP por **cinemática directa**, es decir con los motores apagados. Pero en nuestra aplicacion utilizamos el modo activo, en la que los motores estan encendidos.

Para el proyecto adoptamos **DH estándar** en **metros** y demostramos, además, cinemática inversa para planificar vías que el operario puede seguir o que un sistema actuado podría ejecutar.

## Arquitectura mecánica y GDL

Se modela una estructura 6 GDL: tres ejes “de brazo” para posicionamiento grueso y tres “de muñeca” para orientar la herramienta. Esta partición permite mantener la normal del sensor durante barridos, que es donde este tipo de brazo brilla. Un **7.º eje** es habitual cuando se agrega un **escáner láser** o se requieren orientaciones más ricas sin sacrificar ergonomía, pero el alcance del trabajo integrador queda cubierto con 6.

## Sensado, referencia y herramientas

Cada articulación reporta posición angular; el marco de referencia está en la base. El **TCP** puede ser un **palpador táctil** o el **soporte de una cabeza de escaneo**. La precisión práctica depende de tres patas: 

(i) **calibración geométrica** del modelo

(ii) **resolución/ruido** de encoders

(iii) **rigidez** del conjunto. 

Por eso los flujos serios incluyen **compensación de sonda** y pruebas periódicas tipo **esfera/artefacto** para verificar que la CD refleje la realidad. Esta calibración repetitiva es lo que permite asegurar la precisión y validez de las mediciones realizadas, para evitar corrimientos de los motores y articulaciones de sus posiciones esperadas.

## Definicion y tipos de articulaciones




## Modelado geométrico (DH)

El brazo se parametriza con la tabla DH provista (convención estándar). El orden de ejes y los signos de $\alpha_i$ se fijan siguiendo la regla clásica: $z_i$ alineado al eje de la junta $(i)$, $x_i$ hacia el común perpendicular, y $\alpha_i$ como el giro de $x_i$ a $x_{i+1}$. Con esto se obtienen las matrices homogéneas $A_i$ y la pose ${}^{0}T_6$. La **CD** valida la geometría; la **CI** se usa para planear movimientos hacia poses objetivo sin caer en configuraciones mal condicionadas.

![alt text](Pictures/Robot_con_ejes.png)

| $\theta$ | $d$   | $a$   | $\alpha$         |
| ----     | ----- | ----- | -----            |
| $q_1$    | 0.283 | 0     | $-\frac{\pi}{2}$ |
| $q_2$    | 0     | 0.398 | 0                |
| $q_3$    | 0     | 0.213 | 0                |
| $q_4$    | 0     | 0     | $\frac{\pi}{2}$  |
| $q_5$    | 0     | 0     | $\frac{\pi}{2}$  |
| $q_6$    | 0.166 | 0     | 0                |


>_**Nota**: Si bien el robot del proyecto esta basado en el 'Quantum Max ScanArm'  de Faro, el cuál NO posee muñeca esférica (el que se encuentra en la imagen de arriba), el robor propuesto en este proyecto **(robot Scan Arm G11) SI posee muñeca esférica**._


## Workspace, ergonomía y límites

El **espacio de trabajo** se estima barriendo $\mathbf{q}$ dentro de los **límites articulares**, y muestra el espacio en el cual el robot es capaz de trabajar. Este se descompone para mas facil visualizacion en sus planos XZ (Rojo) y XY (azul). 

Los limites de los parámetros articulares, que definen nuestro workspace son:

|Límite|$q_1$|$q_2$|$q_3$|$q_4$|$q_5$|$q_6$|
|-|-|-|-|-|-|-|
|Superior|170|80|140|140|120|360|
|Inferior|-170|-150|-110|-140|-120|-360|

Debido a la naturaleza no actuada del robot elegido, los limites articulares no son provistos por el fabricante, y no se encuentran en su datasheet. Por lo tanto, se eligieron limites articulares conservativos, que permitan recrear los movimientos del robot con fidelidad y sin movimientos inesperados.

La diferencia entre ambas imagenes se deben a la forma en la que se hace volver al efector.



Si se observa la vista superior XY, se puede notar que es capaz de alcanzar practicamente todos los puntos a su alrededor, menos en la dirección opuesta al marco de referencia de base, per esto no nos genera problemas ya que va a ser preferible trabajar en el frente de este. 

Por otro lado, si se observa la vista lateral XZ, vemos que el brazo es capaz de alcanzar su propia base, puntos sobre y detras de si mismo. Esto puede generar problemas si no se tiene cuidado con las trayectorias, ya que una mala combinacion de isntrucciones articulares podrian causar un choque del robot con su base.
En la proyeccion XZ, el alcance real del robot es mayor que el mostrado, pero se realizó una reduccion para mostrar su espacio de trabajo optimo y util.

Analizando las dimensiones, vemos que tiene un alcance máximo de 0.8m, y una distancia minima ideal de 0.2m a altura 0.2m. Además, si se busca trabajr de forma segura, es recomendable no colocarse cerca de los limites del espacio, por lo que sera necesario trabajar con piezas de dimensiones que no excedan los 0.5m sobre el plano XZ.


### WORKSPACE

![Workspace1](Pictures/Workspace1.png)

Si se busca evitar forzar las articulaciones y los motores, se puede trabajar con factor de seguridad, tal que el valro máximo alcanzado por las articulaciones no supere un cierto porcentaje del límite, asegurando que siempre se tenga un margende mobilidad segura. Utilizando un FS de 90%, se obtiene el nuevo workspace

## WORKSPACE CON FS

![Workspace2](Pictures/Workspace2.png)


En la sección del Jacobiano se abordarán las zonas donde se debe evitar trabajar para evitar problemas con la cinematica inversa.



# Cinematica Directa

A partir de la parametrización obtenida mediante el método **Denavit–Hartenberg**, se desarrolló el cálculo de la **cinemática directa** para el robot **ScanArm 6R**, utilizando tanto formulaciones teóricas como herramientas computacionales.

El objetivo fue determinar la posición y orientación del efector final respecto al sistema base a partir de las variables articulares del robot, representadas por el vector:

$$
\mathbf{q} = [q_1, q_2, q_3, q_4, q_5, q_6]
$$

Mediante el producto de las matrices homogéneas de transformación $( {}^{i-1}\!T_i )$, se obtiene la matriz total:

$$
{}^{0}\!T_{6} = {}^{0}\!T_{1} \cdot {}^{1}\!T_{2} \cdot {}^{2}\!T_{3} \cdot {}^{3}\!T_{4} \cdot {}^{4}\!T_{5} \cdot {}^{5}\!T_{6}
$$

Esta matriz describe completamente la **pose del gripper** respecto al sistema base, permitiendo conocer en todo momento la posición y orientación del efector dentro del entorno de trabajo. Para eso se usó la funcion fkine(q) del toolbox de Peter Corke, con eso y conla matriz DH podemos obtener $( {}^{0}\!T_6 )$

En nuestro caso, la cinemática directa se utilizó principalmente para determinar la posición inicial del gripper y su orientación en distintas configuraciones, lo que permitió validar que el modelo DH adoptado era coherente con la geometría real del brazo.  
A través de este análisis, fue posible ubicar con precisión el efector final en el espacio de trabajo y comprobar que el robot podía alcanzar los puntos requeridos sin violar los límites articulares ni las restricciones físicas del modelo.

Además, la cinemática directa nos permitió **determinar el espacio de trabajo efectivo** del ScanArm, ayudando a definir las zonas de alcance y los límites de movimiento de cada articulación.  
Este resultado fue esencial para el diseño del **layout de la celda**, ya que permitió posicionar el robot de forma que optimizara su rango operativo y evitara interferencias con otros elementos del entorno.

La cinematica directa ademas es fundamental para el funcionamiento del robot en modo cooperativo, ya que en este modo se miden las posiciones articulares y el punto de medicion es determinado por la cinematica directa.

Por último, la validación del modelo se realizó comparando los resultados teóricos obtenidos mediante el algoritmo DH con las posiciones estimadas en el modelo 3D del robot.  
Las diferencias observadas fueron mínimas, confirmando la correcta definición de los parámetros y el buen ajuste entre el modelo matemático y la estructura física.  
De esta forma, se garantizó que las trayectorias generadas posteriormente se desarrollaran dentro de los límites seguros del sistema y del espacio disponible en la celda de trabajo.

### Método de solución

1. **Tabla DH**  
   Se utiliza la tabla DH definida en el punto (i), con los parámetros $(a_i, \alpha_i, d_i, \theta_i)$.  
   Para las articulaciones revolutas, el parámetro variable es el ángulo $(\theta_i = q_i + \theta_{i0})$.

2. **Transformación elemental**  
   Cada matriz ${}^{i-1}T_i$ se construye según:

   $$
   {}^{i-1}\!T_i =
   \begin{bmatrix}
   \cos\theta_i & -\sin\theta_i\cos\alpha_i & \sin\theta_i\sin\alpha_i & a_i\cos\theta_i \\
   \sin\theta_i & \cos\theta_i\cos\alpha_i & -\cos\theta_i\sin\alpha_i & a_i\sin\theta_i \\
   0 & \sin\alpha_i & \cos\alpha_i & d_i \\
   0 & 0 & 0 & 1
   \end{bmatrix}
   $$

3. **Composición de transformaciones**  
   La matriz total $( {}^{0}\!T_{6} )$ se obtiene multiplicando secuencialmente las seis matrices elementales.

4. **Obtención de la pose**  
   - **Posición:** coordenadas del efector $(x, y, z)$ = última columna de $( {}^{0}\!T_{6} )$.  
   - **Orientación:** matriz de rotación $( R = {}^{0}\!R_{6} )$.




# Cinematica Inversa

Para el calculo de la cinematica inversa de nuestro robot utilizamos la funcion de cinematica inversa cin_inv_Faro que definimos el el trabajo practico N° 5B, que utiliza el calculo analitico para encontrar los $q_1$, $q_2$, $q_3$, $q_4$, $q_5$ y $q_6$ para llegar al punto inicial de la trayectoria y luego a los siguientes puntos de la misma.

Es importante destacar que pudimos realizar el desacople de la muñeca del resto del robot por el hecho de haber considerado que nuestro robot es del tipo muñeca esferica, sin esta simplificacion no podriamos haber utilizado este metodo. 

Llamaremos $p$ al punto al que queremos llegar, $p_m$ al punto del sistema de la muñeca, en un principio referenciados al sistema $S_0$ (base de nuestro robot)

+  Para calcular $\mathbf{q_1}$ : como rota el brazo entero, y sabemos que la muñeca se debe alinear con el punto $p$, entonces basta con calcular  $q_1 = \tan\left(\frac{p_{my}}{p_{mx}}\right)$ (2 soluciones)
+ Para calcular $\mathbf{q_2}$ nos trasladamos el sistema a $S_1$ y referenciamos todo al mismo, entonces para cada $q_1$, donde se puede calcular 2 valores diferentes de $q_2$ (codo arriba y codo abajo):

![alt text](<Pictures/grafico Q2.png>)   

 de donde calculamos $r$, $G$, $B$ y $q_2= B \pm G$

+ Para el calculo de $q_3$ volvemos a arefenrenciar todo al sistema $S_2$ y calculamos $q_3$ con las proyecciones de $p_m$

+ Para el calculo de $q_4$, $q_5$ y $q_6$ se construye la matriz de transformacion directa ${}^{0}T_6$  y obetenemos la transformada relativa muñeca ${}^{3}T_6$ necesaria para alcanzar T, como la muñeca solo nos aporta la orientacion, usamos solo la parte $\mathbb{R}$ de ${}^{3}T_6$, y calculamos $q_4$, $q_5$ y $q_6$.
  
Para verificar que el robot llegue al punto indicado con los $q$ calculados usamos la funcion `fkine` para verificar que llegamos a la matriz T del punto, o simplemente comparamos que los $q$ coincidan. Depende de que parametro de entrada le demos a la funcion.


Ademas, en el calculo de la cinemtica inversa, nuestro robot llega al punto indicado, pero el $q_2$ esta $2\pi$ adelantado, esto es debido al calculo trigonometrico ya que las soluciones matematicas son infinitcas, por lo que tuvimos que restarle $2\pi$, para asegurarnos de que los valores de q2 esten dentro de los limites articulares.


# Relacion de velocidades  (Jacobiano)

El Jacobiano es una transformación matemática que permite relacionar las velocidades en el espacio articular con las velocidades lineales y angulares en el espacio cartesiano. Además, nos permite evaluar puntos donde el robot puede fallar hallando los puntos singulares, aquellos donde el determinante del Jacobiano se vuelven 0. Ademas, si se conocen las fuerzas aplicadas sobre el efector final, nos permite encontrar los torques aplicados sobre las articulaciones, aunque esto ultimo no sera desarrollado en detalle en este informe. Esto se representa como:

$$
\begin{bmatrix}
\dot{\mathbf{p}}\\[2pt]
\boldsymbol{\omega}
\end{bmatrix}
=
J(q)\,\dot{\mathbf{q}}
\qquad\text{y}\qquad
\boldsymbol{\tau}=J(q)^{\!\top}\!
\begin{bmatrix}
\mathbf{F}\\[2pt]\mathbf{M}
\end{bmatrix}
$$

Siendo que nuestro robot posee 6 articulaciones, y ademas 6 grados de libertad, su Jacobiano tiene dimension 6x6. Este lo descompondremos en 2 partes, una de velocidades lineales ($J_v$), y otra de velocidades angulares ($J_\omega$), ambas de 3x6, tal que el Jacobiano completo resulte: 

$$
J(q)=
\begin{bmatrix}
J_v^{(\text{brazo})} & J_v^{(\text{muñeca})}\\[4pt]
J_\omega^{(\text{brazo})} & J_\omega^{(\text{muñeca})}
\end{bmatrix}
,\qquad
J_v,J_\omega \in \mathbb{R}^{3\times6}.
$$


Utilizaremos ademas las siguientes abreviaciones:

$$
\begin{aligned}
s_i &= \sin(q_i), \qquad &c_i &= \cos(q_i), &\text{para } i = 1,\dots,6,\\[4pt]
s_{23} &= \sin(q_2 + q_3), \qquad &c_{23} &= \cos(q_2 + q_3),\\[4pt]
s_{234} &= \sin(q_2 + q_3 + q_4), \qquad &c_{234} &= \cos(q_2 + q_3 + q_4).
\end{aligned}
$$


### Parte lineal ($J_v$)


$$
J_v^{(\text{brazo})} =
\begin{bmatrix}
-\;a_2 s_1 - a_3 \sin(q_1{+}q_2) &
c_1\,d_6\,\cos q_5\,s_{234} &
c_1\,d_6\,\cos q_5\,s_{234}\\[6pt]
\;\;a_2 c_1 + a_3 \cos(q_1{+}q_2) &
s_1\,d_6\,\cos q_5\,s_{234} &
s_1\,d_6\,\cos q_5\,s_{234}\\[6pt]
0 &
a_2 + a_3 \cos q_2 + d_6\,c_{234}\cos q_5 &
a_3 \cos q_2 + d_6\,c_{234}\cos q_5
\end{bmatrix},
$$

$$
J_v^{(\text{muñeca})} =
\begin{bmatrix}
c_1\,d_6\,\cos q_5\,s_{234} &
-\,d_6\big(s_1 \cos q_5 + c_1 c_{234}\sin q_5\big) &
0\\[6pt]
s_1\,d_6\,\cos q_5\,s_{234} &
-\,d_6\big(s_1 c_{234}\sin q_5 - c_1 \cos q_5\big) &
0\\[6pt]
d_6\,c_{234}\cos q_5 &
d_6\,s_{234}\sin q_5 &
0
\end{bmatrix}.
$$


### Parte angular ($J_\omega$)

$$
J_\omega^{(\text{brazo})} =
\begin{bmatrix}
0 & s_1 & s_1\\[4pt]
0 & -c_1 & -c_1\\[4pt]
1 & 0 & 0
\end{bmatrix},
\qquad
J_\omega^{(\text{muñeca})} =
\begin{bmatrix}
s_1 & c_1 s_{234} & -s_1\sin q_5 + c_1 c_{234}\cos q_5\\[4pt]
-\,c_1 & s_1 s_{234} & s_1 c_{234}\cos q_5 + c_1 \sin q_5\\[4pt]
0 & c_{234} & -\cos q_5\,s_{234}
\end{bmatrix}.
$$


### Determinante del Jacobiano

Aprovechando el desacople por la muñeca esférica $(J = J_{\text{brazo}} \oplus J_{\text{mu\~neca}})$:

$$
\boxed{
\det(J) = \det(J_v^{\text{brazo}})\,\det(J_\omega^{\text{mu\~neca}})
}
$$

y, para una muñeca esférica de tres ejes concurrentes:

$$
\boxed{
\det(J_\omega^{\text{mu\~neca}}) = \sin(q_5)
}
$$

Debido al offset de 90º aplicado a $q_5$, $sin(q_5)$ pasa a ser $cos(q_5)$ por identidad trigonometrica

Por lo tanto, el determinante total del Jacobiano geométrico del efector final es:

$$
\boxed{
\det(J_{\text{ef}}) = \cos(q_5)\,\det(J_v^{\text{brazo}})
}
$$

donde $\det(J_v^{\text{brazo}})$ depende sólo de la geometría del brazo. La formula final del determinante resulta:

$$
\det J(q)
=
\cos(q_5)\,\Big[
\underbrace{0.033740052}_{\large \frac{8435013}{250000000}}\;\cos q_2\,\sin q_3
\;-\;
\underbrace{0.018056862}_{\large \frac{9028431}{500000000}}\;\sin q_2
\;+\;
\underbrace{0.018056862}_{\large \frac{9028431}{500000000}}\;\cos q_3\,\sin q_2
\;+\;
\underbrace{0.018056862}_{\large \frac{9028431}{500000000}}\;\cos q_2\,\cos q_3\,\sin q_3
\Big].
$$

Y de forma simplificada:

$$
\det J(q)
=
\cos(q_5)\,\Big[
\cos q_2\,\sin q_3\;\Big(\,0.033740052 + 0.018056862\,\cos q_3\Big)
\;+\;
0.018056862\;\sin q_2\;\big(\cos q_3-1\big)
\Big].
$$



Donde podemos observar que $q_1$ no tiene influencia sobre el determinante del brazo, solo $q_2$, $q_3$, $q_5$ y las dimensiones del robot.


### Puntos singulares

Desarrollando las formulas, llegamos a:


$$
\det J_{\mathrm{ef}}(q)=0
\;\Longleftrightarrow\;
\begin{cases}
\sin(q_3)=0 &\Rightarrow\; q_3=0,\ \pi \quad \text{(Codo estirado/plegado)  (Caso 1)}\\[4pt]
\cos(q_5)=0 &\Rightarrow\; q_5=\tfrac{\pi}{2},\ \tfrac{3\pi}{2} \quad \text{(Muñeca alineada)  (Caso 2)}\\[4pt]
\text{$O_c$ = (0,0,Z); (Muñeca sobre base)  (Caso 3)}
\end{cases}
$$


Observamos que hay 3 casos donde det($J$) se vuelve 0, los cuales definen cada uno un tipo de singularidad que se debe evitar.

El primer caso sucede cuando el brazo se encuentra completamente estirado (o flexionado, pero debido a los limites articulares esto no es posible). Su solución es sencilla: reducir ligeramente el área de trabajo, lo cual además evita crear torques excesivos sobre el brazo.

El segundo caso sucede cuando la muñeca del robot se ubica a 90ª respecto del antebrazo de este. Causa que los ejes Z de q_4 y q_6 se alineen, lo que elimina un grado de libertad. Esto se puede solucionar trabajando con unos limites articulares más reducidos, y evitando doblar la muñeca hacia "adentro".

El tercer y ultimo caso es causado cuando la muñeca $(O_c)$ se ubica sobre el eje Z de la base (coordendas $0,0$). Esto depende de una combinación de valores de los parametros articulares $q_2$ y $q_3$, por lo que no hay una única limitación articular que lo solucione. Pero, se puede implementar una verificación en el espacio cartesiano que marque a la posición como no válida. Además esto se soluciona facilmente al ver que el espacio de trabajo no incluirá tal zona, ya que implicaría trabajar sobre el robot o directamente en colisión con este.


# Planificacion y generacion de trayectoria

La planificación de trayectorias en robótica parte de tres piezas conceptuales: **la tarea**, **la representación geométrica** y **las restricciones**. La tarea define qué debe hacer el efector (posicionar, seguir una curva, inspeccionar una zona, evitar un volumen). La representación geométrica especifica dónde debe hacerlo: un camino en el espacio articular $q$ o en el espacio de tarea $SE(3)$ mediante poses objetivo y tolerancias. Las restricciones delimitan lo físicamente ejecutable: cotas de junta $(q_{\min},q_{\max})$, límites de velocidad y aceleración $(\dot q_{\max},\ddot q_{\max})$, restricciones de orientación, zonas prohibidas y márgenes de seguridad.

Conviene distinguir **camino** y **trayectoria**. El camino es puramente geométrico (una curva en $q$ o en $SE(3)$ sin noción de tiempo). La trayectoria es el camino **temporalizado** por una ley $t\mapsto s(t)$ que fija posición, velocidad y aceleración en cada instante. En práctica, se propone primero un camino que cumpla la tarea y evite colisiones y, luego, se aplica una **parametrización temporal** que respete límites y suavidad. Esta temporalización puede ser polinómica (quintic con condiciones en extremos), tipo LSPB (trapezoidal), S-curve con acotación de jerk, o cualquier ley que garantice continuidad al orden requerido por el actuador y el sensor.

Si el camino se define en espacio de tarea, cada muestra $T(s)$ se proyecta a juntas resolviendo cinemática inversa con semillas continuas para preservar suavidad y minimizar saltos. La factibilidad cinemática se contrasta evaluando el jacobiano $J(q)$: la relación $\dot x = J(q) \, \dot q$ permite estimar $\dot q$ inducida por la velocidad deseada en tarea y detectar saturaciones antes de ejecutar. El condicionamiento de $J$ guía el **evitado de singularidades** y las re-orientaciones del efector cuando correspondan. Si el camino se define directamente en espacio articular, la verificación es análoga pero sin resolver IK, y el control de colisiones/alcanzabilidad se hace mapeando a (SE(3)) mediante cinemática directa.

## Precisión y repetibilidad: qué afecta y cómo se cuida

Las fuentes de error típicas incluyen **offsets geométricos**, **backlash**, **compliance**, **deriva térmica** y el propio **ruido de medida**. En análisis, el **jacobiano** permite entender cómo pequeños errores articulares se amplifican en el TCP. En práctica, se atenúa con buena calibración, perfiles de movimiento suaves (LSPB/quintic) y evitando regiones cercanas a singularidades.

## Modos de uso en el proyecto

En **modo pasivo (realista)** el operador guía el brazo siguiendo las **trayectorias recomendadas** generadas por el planificador. El sistema registra poses vía **cinemática directa** a partir de las lecturas de junta y la nube resultante se referencia al marco de la placa. La utilidad de planificar no desaparece: el raster aporta repetibilidad, uniformidad de espaciamiento y un marco para comparar ejecuciones y detectar desviaciones.

En la **variante cooperativa actuada (académica)**, el mismo pipeline se ejecuta automáticamente: la pasada recta se sintetiza en cartesiano y se proyecta a juntas con cinemática inversa; los u-turns se generan mediante waypoints y se interpolan en juntas, todo bajo límites de posición, velocidad y aceleración. El sistema puede trabajar en control de admitancia para permitir guía humana sin perder la capacidad de **seguir perfiles**. En ambos modos las **demostraciones** se circunscriben a casos canónicos y reproducibles (pasadas rectas, arcos simples y raster plano), suficientes para validar el modelo, ilustrar el papel del jacobiano y documentar con claridad los resultados de simulación.


## Muestreo de una placa de dimensiones $H \times W \times e$
El pipeline de planificación se implementa como un programa monolítico que genera la trayectoria completa, tanto las líneas de escaneo como las transiciones, de forma secuencial y acumulativa. La estrategia es definir todos los movimientos en el espacio cartesiano y resolverlos punto a punto mediante cinemática inversa.
El programa itera por el número de líneas de escaneo (num_lines). En cada iteración:

### Línea de Escaneo:

Se definen los puntos cartesianos de inicio `P0` y fin `Pf` de la línea. La dirección de avance en X se alterna para líneas pares e impares.

Se calcula la duración total del segmento `T_total` basándose en una velocidad cartesiana constante `vel_scan`. Esto genera un perfil de velocidad rectangular, no trapezoidal.

Se genera una trayectoria de poses cartesianas T_path usando $ctraj(T0, Tf, N)$, que interpola linealmente desde la pose de inicio a la de fin.

Se genera la trayectoria articular `Q_seg` iterando sobre cada pose `Tk` de `T_path` y resolviendo la cinemática inversa (`ikine` o `ikcon`). Para asegurar continuidad, la solución articular previa q_prev se usa como semilla `q0` para la siguiente iteración.

### Transición (U-Turn):

Si no es la última línea, la transición (maniobra en "U") no se genera como un movimiento único y suave en juntas, sino como tres segmentos cartesianos rectos e independientes:

Elevar P_lift: Un movimiento recto en +Z.

Desplazar P_shift: Un movimiento recto en +Y.

Descender P_descend: Un movimiento recto en -Z hasta la posición inicial de la siguiente línea.

Cada uno de estos tres segmentos se genera usando exactamente el mismo método que la línea de escaneo: se calcula un `T_total` (basado en vel_move), se genera un `T_path` con `ctraj`, y se resuelve la cinemática inversa punto a punto en un bucle.

Todos los segmentos articulares `Q_seg` se concatenan en el arreglo `Q_all` y sus vectores de tiempo en `t_all`. Finalmente, las velocidades `Qd` y aceleraciones `Qdd` articulares no se planifican explícitamente, sino que se calculan a posteriori de forma numérica (usando gradient()) a partir de la trayectoria de posición Q_all resultante.
El programa calcula y grafica:

- $(Q,\dot Q,\ddot Q)$ (articular);

- $(\dot x,\ddot x)$ (TCP) vía $(J)$ y $(\dot J)$;

- $(\kappa(J))$ y $(w)$ (manipulabilidad);

- **Traza del TCP** en `R.plot(...,'trail','r-')`.

## Gráficos obtenidos

![alt text](<Pictures/Perfil de Posicion Cartesiana.png>)

La **posición cartesiana** confirma el esquema de raster: en cada pasada el eje $x(t)$ evoluciona **monótonamente** con tramos lineales que corresponden al crucero a velocidad casi constante, separados por breves fases de aceleración y frenado; el eje $y(t)$ permanece **constante** durante la medición y cambia por **escalones** de $\Delta y$ únicamente en los retornos; el eje $z(t)$ se mantiene en la cota de escaneo $z_0$ y solo realiza el **ciclo de despeje** (subir–cruzar–bajar) entre líneas. La trayectoria resultante es continua en posición y, dentro de cada pasada, también en velocidad, lo que garantiza un espaciamiento uniforme de muestras sobre la superficie medida y separa claramente los tramos útiles de las transiciones.


![alt text](<Pictures/Velocidad Cartesiana X.png>)

![alt text](<Pictures/Velocidad Cartesiana Y.png>)

![alt text](<Pictures/Velocidad Cartesiana Z.png>)

![alt text](<Pictures/Modulo Velocidad Cartesiana.png>)

Los perfiles de **velocidad** muestran que las pasadas de escaneo en $x$ se realizan con **velocidad prácticamente constante**: $v_x$ presenta mesetas cercanas a $0.075\text{–}0.08\ \mathrm{m/s}$ y el **módulo** $\lVert \mathbf{v}\rVert$ replica el mismo valor durante los tramos activos. Esto confirma que la temporalización adoptada garantiza avance uniforme sobre cada línea, condición clave para muestreo homogéneo y para evitar sesgos por variación de velocidad.

Las **transiciones entre líneas (U-turn)** se manifiestan como pulsos bien definidos en $v_y$ y, sobre todo, en $v_z$: breve elevación, desplazamiento lateral y descenso. La separación temporal de estos pulsos coincide con el fin de cada pasada en $x$, lo que respalda que el plan se está ejecutando según el pipeline propuesto (elevar-mover-bajar). La duración corta de estas maniobras reduce el tiempo improductivo entre pasadas, pero concentra los cambios de estado en ventanas pequeñas.

![alt text](<Pictures/Perfil de Aceleracion Cartesiana.png>)

En **aceleración**, se observa el comportamiento esperado de un perfil tipo LSPB: $a_x$ es casi por tramos constante durante aceleración y frenado, y nula en el crucero. En cambio, $a_z$ exhibe picos angostos que superan a los de $a_x$ y $a_y$ por un orden de magnitud (del orden de $0.6\ \mathrm{m/s^2}$ según la escala de la figura$, coherentes con el “salto” vertical de los U-turn. Estos picos no afectan el muestreo en los tramos de escaneo, pero sí **incrementan la exigencia dinámica** en las articulaciones durante la transición.

Desde el punto de vista de **calidad metrológica**, los gráficos validan dos puntos: i) el **avance constante** en las pasadas, que asegura espaciamiento uniforme de muestras; ii) **transiciones discretas** fuera de la zona de medición, por lo que las variaciones de $\mathbf{v}$ y $\mathbf{a}$ no contaminan los datos útiles. Para robustecer aún más el lazo, conviene: suavizar las transiciones en $z$ con **perfil S-curve o quintic** (limitando jerk), usar una **velocidad de movimiento** $v_\mathrm{move}$ menor que $v_\mathrm{scan}$ en las fases de elevar/bajar, y añadir un **pequeño dwell** antes de iniciar cada nueva pasada para disipar transitorios.

![alt text](Pictures/Jacobian_conditioning.png)

Finalmente, las variables articulares tienen gráficos:

![alt text](<Pictures/Perfil de posicion Q.png>)

![alt text](<Pictures/Perfil de velocidad Qd.png>)

![alt text](<Pictures/Perfil de aceleracion Qdd.png>)


# Sensores y actuadores

En un Scan Arm de grado metrológico el operario es el motor. La arquitectura base es pasiva: cada articulación incorpora sensado angular y el sistema reconstruye la pose por  cinemática directa . “Actuadores”, en este contexto, no significa servomotores que mueven el brazo, sino  mecanismos y ayudas activas que mejoran ergonomía, seguridad y calidad de dato. Para cumplir con la cátedra también contemplamos una variante cooperativa actuada en simulación, donde la CI cobra sentido operativo.

## Sensores

Clasificamos los sensores del sistema en internos (aquellos que miden el estado propio del robot) y externos (aquellos en el efector final, dedicados a medir la tarea).

### Sensores Internos (Estado Articular y Cinemática)

Este es el conjunto de sensores más críticos para el análisis cinemático (CD/CI) del robot. El sistema se basa en una serie de codificadores angulares (encoders) de alta resolución, montados en el eje de cada una de las 6 articulaciones (GDL).
La elección de encoders absolutos es deliberada y fundamental para la metrología. A diferencia de otros tipos, estos reportan la posición angular real de la junta desde el momento del encendido, sin necesidad de un ciclo de "homing" (búsqueda de cero). Esto garantiza que la referencia global del brazo sea robusta ante, por ejemplo, un corte de energía, que de otra manera invalidaría una medición en curso.

Estos sensores son la fuente de datos primaria que conecta el modelo geométrico (DH) con el mundo físico. Miden con precisión el ángulo de cada junta, proporcionando en tiempo real el vector de variables articulares:

$$q = [q_1, q_2, q_3, q_4, q_5, q_6]^T$$

El rol de estos sensores para este proyecto:

* Rol en la Cinemática Directa (CD): En la operación normal (pasiva), donde el operario mueve el brazo, los valores leídos de estos encoders son la entrada directa al modelo de Cinemática Directa. El software utiliza este vector q para resolver la ecuación ${}^{0}T6 = f(q)$ y así calcular la pose (posición y orientación) del efector final (TCP) en el espacio.

* Rol en la Cinemática Inversa (CI): En la "variante cooperativa actuada" (evaluada académicamente en este informe), la CI calcula el vector q deseado para alcanzar una pose objetivo. Los encoders se convierten entonces en la señal de retroalimentación para el lazo de control de posición de cada motor, asegurando que el robot alcance y mantenga la configuración articular calculada.

### Sensores Externos (Adquisición de Tarea)

Estos sensores se ubican en el efector final (TCP) y su función es digitalizar la pieza de trabajo. Su propia precisión depende de que el modelo de CD, alimentado por los sensores internos, sea correcto.

* Palpador Táctil (Touch Probe): Es un sensor de contacto discreto. Al tocar la superficie de la pieza, genera un trigger (disparo) eléctrico de alta precisión.. En ese instante, el sistema "congela" la lectura de los encoders internos y calcula la CD para registrar la coordenada 3D exacta de ese único punto.

* Escáner Láser (LLP): Es un sensor de adquisición masiva. Utiliza triangulación óptica (una línea láser proyectada y una cámara CMOS) para capturar millones de puntos (nube de puntos) por segundo . La calidad de la nube de puntos depende de factores de la superficie (reflectividad, ángulo de incidencia) y, crucialmente, de la precisión y velocidad con que el sistema calcula la pose del escáner (CD) en cada instante, basándose en los encoders de las juntas.Para el TP alcanza con describir la física, el pipeline de registro y los límites: reflectividad, ángulo de incidencia, halo especular.


## “Actuadores” en brazos pasivos

En un Scan Arm de grado metrológico, el operario es el "motor" primario. Por lo tanto, en el contexto pasivo (su uso industrial más habitual), el término "actuador" no se refiere a servomotores que mueven el brazo, sino a los mecanismos que asisten al operario y garantizan la calidad de la medición.
Los componentes clave son:

* Contrapeso Interno: Un mecanismo pasivo (resortes o masas) que neutraliza la gravedad. Esto reduce la fatiga del operario y evita que el brazo caiga si se suelta, permitiendo mediciones más estables.

* Bloqueo/Freno de Junta: Sistemas de sujeción (a menudo electromagnéticos) que permiten al operario fijar una o todas las articulaciones en una pose específica. Esto es útil para soltar las manos, cambiar la sonda o parquear el brazo con seguridad.

* Retroalimentación al Usuario: Actuadores hápticos (vibración) o indicadores visuales en la muñeca. Alertan al operario sobre eventos clave: confirmación de un punto táctil, baja calidad de escaneo (si se usa LLP) o si se está alcanzando un límite articular.


## Variante cooperativa actuada (enfoque académico)

Para justificar cinemática inversa y planeamiento, se plantea una opción actuada en simulación:

* Motores backdrivable o con control de admitancia/impedancia para que el humano guíe y el robot asista, con perfiles LSPB/quintic y límites de junta.

* Sensado de par (o estimado) para contacto suave y virtual fixtures que “peguen” la punta a una superficie virtual durante el barrido.

* Seguridad: paradas controladas, zonas de exclusión y validación previa del Jacobiano para evitar regiones mal condicionadas.


## Selección concreta para el proyecto

Para los fines de este informe y sus simulaciones en MATLAB, se adopta la siguiente configuración:
* Sensores Internos (CD/CI): Se asume el uso de encoders absolutos en cada junta, como "mejor práctica" por su robustez ante cortes de energía. Son la fuente de q para todos los cálculos.

* Sensores Externos (Tarea): Se utiliza el palpador táctil como baseline para la validación de la Cinemática Directa. Se describe el escenario de escáner láser (LLP)  para justificar la necesidad de control de orientación y el análisis de trayectorias cartesianas.

* Actuación (Simulación): El análisis de CI, Jacobiano y planificación de movimiento se realiza sobre la variante cooperativa actuada. Las simulaciones en MATLAB ejecutarán trayectorias P2P (LSPB/tpoly) y barridos cartesianos, respetando los límites articulares definidos y demostrando la viabilidad de los modelos cinemáticos desarrollados.



# Conclusiones

El proyecto estableció una base cinemática sólida para un brazo tipo ScanArm de 6 GDL parametrizado con DH estándar (unidades en metros), alineada con los objetivos de la cátedra. Se modeló la cadena geométrica, se implementó la cinemática directa y se incorporó una solución analitica de cinemática inversa orientada a planificación, aun considerando que el uso industrial habitual de estos equipos es pasivo. Este enfoque dual permitió estudiar alcanzabilidad y orientar la herramienta con criterios reproducibles, sin depender de un hardware actuado real.

El análisis del jacobiano y de las singularidades brindó un mapa claro de zonas mal condicionadas, útil para definir restricciones de operación y para justificar decisiones de trayectoria. La planificación en espacio articular con perfiles LSPB y polinomios de quinto orden, junto con trayectorias cartesianas en SE(3), mostró transiciones suaves que respetan límites de posición, velocidad y aceleración, condición necesaria para preservar la calidad metrológica. Las demostraciones se realizaron deliberadamente con trayectorias simples (punto a punto, arco con normal controlada y raster sobre plano), suficientes para validar el modelo, ilustrar la relación con el jacobiano y documentar resultados de forma clara.

Además, el estudio diferencial del jacobiano permitió cuantificar la relación entre velocidades en el espacio de tarea y en el espacio articular. A partir de $\dot{\mathbf{x}}=J(\mathbf{q}),\dot{\mathbf{q}}$ se utilizó $\dot{\mathbf{q}}=J^{-1}(\mathbf{q}),\dot{\mathbf{x}}_{\mathrm{ref}}$ para imponer una **velocidad constante del TCP** a lo largo de las pasadas y dimensionar en qué medida deben modificarse las **coordenadas articulares y sus derivadas** para sostener esa consigna. Este enfoque facilitó detectar con anticipación saturaciones de velocidad/acceleración por junta y ajustar la temporalización de los perfiles antes de ejecutar la trayectoria.

En cuanto a instrumentación, se describió el rol central de los encoders de junta y se caracterizaron dos modalidades de adquisición en el efector (palpador y escáner óptico), subrayando cómo la calidad de dato depende de la calibración geométrica, la rigidez del conjunto y los perfiles cinemáticos. Para el contexto académico, se presentó una variante cooperativa actuada a nivel de simulación que habilita la ejecución automática de vías mediante cinemática inversa, sin alterar el modelo geométrico de base ni la trazabilidad del análisis.

Por otra parte, el hecho de contar con una **muñeca esférica** permitió abordar la cinemática inversa mediante la descomposición clásica de **posición** (primeros tres grados de libertad hacia el centro de muñeca) y **orientación** (tres últimos grados de libertad), habilitando soluciones analíticas robustas y selección de ramas que evitan configuraciones desfavorables. En contraste, para un 6R **sin muñeca esférica** la CI debe resolverse con métodos numéricos y/o geométricos acoplados; en nuestras pruebas conceptuales, lograr simultáneamente una **velocidad de avance constante** y una **orientación estrictamente normal** a la superficie incrementó de forma notable la complejidad y la sensibilidad del planificador.


El trabajo es reproducible: se integró en scripts de MATLAB que generan las figuras de workspace, los perfiles cinemáticos y las animaciones empleadas en el informe. Esto facilita repetir experimentos, ajustar límites articulares y explorar nuevas trayectorias de forma controlada.

Como limitaciones, no se realizó calibración geométrica con artefactos físicos ni validación de incertidumbre de medida en banco real, y no se integró control a nivel de par ni sensado de fuerza para contacto. Quedan abiertas líneas de trabajo que agregan valor directo: calibración y compensación geométrica, incorporación de un modelo simple de error para estimar incertidumbre en el TCP, extensión a 7 GDL para mejorar la orientabilidad en escaneo, y validación experimental de perfiles temporales frente a restricciones de sensor (frecuencia de muestreo, reflectividad y ángulo de incidencia). Estas extensiones no modifican los resultados centrales del informe: el modelo cinemático es consistente, las trayectorias cumplen criterios de suavidad y límites, y el marco propuesto es adecuado para inspección y escaneo en una célula didáctica o de baja complejidad.


# Referencias

[1] FARO Technologies, “Quantum X FaroArm® Series | Brochure,” Oct. 2024. [Online]. Available: https://…/Brochure_Quantum-X_3DM_ENG_Web.pdf. ([letoltes.sidex.hu][1])

[2] FARO Technologies, “Quantum X FaroArm® Series (product page),” 2025. [Online]. Available: [https://www.faro.com/en/Products/Hardware/Quantum-FaroArms](https://www.faro.com/en/Products/Hardware/Quantum-FaroArms). ([FARO.com][2])

[3] Hexagon Manufacturing Intelligence, “The Absolute Arm (A4 brochure),” 2025. [Online]. Available: https://…/Hexagon-MI-Arm-Absolute-Arm-Brochure-A4-en.pdf. ([shop.hexagonmi.com][3])

[4] Hexagon Manufacturing Intelligence, “Absolute Arm systems and accessories (catalog),” 2023. [Online]. Available: [https://hexagon.com/…/hexagon_mi_arm_absolute_arm_catalog_23_210x210_en.pdf](https://hexagon.com/…/hexagon_mi_arm_absolute_arm_catalog_23_210x210_en.pdf). ([hexagon.com][4])

[5] Nikon Metrology, “MCAx S portable CMM arm with ModelMaker H120 (datasheet),” 2021. [Online]. Available: [https://industry.nikon.com/…/mcaxs-modelmaker-h120-en.pdf](https://industry.nikon.com/…/mcaxs-modelmaker-h120-en.pdf). ([Nikon Industrial Metrology - MASTER][5])

[6] Kreon, “ACE measuring arm & Skyline scanners (brochure),” 2021. [Online]. Available: [https://www.crossco.com/…/Kreon-Ace-Brochure.pdf](https://www.crossco.com/…/Kreon-Ace-Brochure.pdf). ([CrossCo][6])

[7] Kreon, “Measuring arm – Ace (product page),” 2025. [Online]. Available: [https://www.kreon3d.com/products/ace-measuring-arm](https://www.kreon3d.com/products/ace-measuring-arm). ([kreon3d.com][7])

[8] Creaform, “HandyPROBE technical specifications,” 2025. [Online]. Available: [https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe/technical-specifications](https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe/technical-specifications). ([creaform3d.com][8])

[9] Creaform, “Coordinate measuring machines – HandyPROBE (product page),” 2025. [Online]. Available: [https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe](https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe). ([creaform3d.com][9])

[10] ISO, *ISO 10360-12:2016 — Geometrical product specifications (GPS) — Acceptance and reverification tests for coordinate measuring machines (CMM) — Part 12: Articulated arm CMM*. Geneva, Switzerland: International Organization for Standardization, 2016. [Online]. Available: https://…/ISO-10360-12-2016.pdf. ([Iteh][10])

[11] ISO, *ISO 10360-8:2013 — Geometrical product specifications (GPS) — Acceptance and reverification tests for coordinate measuring machines (CMM) — Part 8: CMMs with optical distance sensors*. Geneva, Switzerland: International Organization for Standardization, 2013. [Online]. Available: [https://www.iso.org/standard/54522.html](https://www.iso.org/standard/54522.html). ([iso.org][11])

[12] Kreon, “ISO 10360-12 standard for measuring arms (explainer article),” 2023. [Online]. Available: [https://www.kreon3d.com/article/iso-10360-12-standard-for-measuring-arms](https://www.kreon3d.com/article/iso-10360-12-standard-for-measuring-arms). ([kreon3d.com][12])

[13] P. Corke, *Robotics, Vision & Control: Fundamental Algorithms in MATLAB*, 2nd ed. Cham, Switzerland: Springer, 2017. [Online]. Available: [https://petercorke.com/books/robotics-vision-control-all-versions/](https://petercorke.com/books/robotics-vision-control-all-versions/). ([Peter Corke][13])

[14] P. Corke, “Robotics Toolbox for MATLAB (official site),” 2025. [Online]. Available: [https://petercorke.com/toolboxes/robotics-toolbox/](https://petercorke.com/toolboxes/robotics-toolbox/). ([Peter Corke][14])

[15] P. Corke, “Robotics Toolbox for MATLAB (GitHub repository),” 2025. [Online]. Available: [https://github.com/petercorke/robotics-toolbox-matlab](https://github.com/petercorke/robotics-toolbox-matlab). ([GitHub][15])

[16] J. Haviland and P. Corke, “A Systematic Approach to Computing the Manipulator Jacobian and Hessian using the Elementary Transform Sequence,” *arXiv:2010.08696*, 2020. [Online]. Available: [https://arxiv.org/abs/2010.08696](https://arxiv.org/abs/2010.08696). ([arXiv][16])

[17] J. Haviland and P. Corke, “Manipulator Differential Kinematics: Part 1: Kinematics, Velocity, and Applications,” *arXiv:2207.01796*, 2022. [Online]. Available: [https://arxiv.org/abs/2207.01796](https://arxiv.org/abs/2207.01796). ([arXiv][17])

[18] FARO Technologies, “Quantum X FaroArm® Series | Brochure (web page),” 2025. [Online]. Available: [https://www.faro.com/en/Resource-Library/Brochure/Quantum-FaroArm](https://www.faro.com/en/Resource-Library/Brochure/Quantum-FaroArm). ([FARO.com][18])



[1]: https://letoltes.sidex.hu/metrologia/faro-quantum-x-brochure-eng.pdf?utm_source=chatgpt.com "Quantum X - FaroArm® Series"
[2]: https://www.faro.com/en/Products/Hardware/Quantum-FaroArms?utm_source=chatgpt.com "Quantum X FaroArm® Series"
[3]: https://shop.hexagonmi.com/medias/Hexagon-MI-Arm-Absolute-Arm-Brochure-A4-en.pdf?context=bWFzdGVyfHJvb3R8NTczMDAzN3xhcHBsaWNhdGlvbi9wZGZ8YURVekwyaGtOUzg1TkRjeE1ERTFOVGd6TnpjMEwwaGxlR0ZuYjI1ZlRVbGZRWEp0WDBGaWMyOXNkWFJsSUVGeWJWOUNjbTlqYUhWeVpWOUJORjlsYmk1d1pHWXw3ZDA0NjhlN2I3ZDRlOGJkY2M1MjFmMDA5OGE0ZGRhZjUwNWQwNDkxMGMxNDljYjBiNmQzZWM5YjFiNjc2YTNk&utm_source=chatgpt.com "The Absolute Arm - Shop Hexagon"
[4]: https://hexagon.com/-/media/project/one-web/master-site/mi/pdf/hexagon_mi_arm_absolute_arm_catalog_23_210x210_en.pdf?utm_source=chatgpt.com "Absolute Arm systems and accessories"
[5]: https://industry.nikon.com/en-gb//wp-content/uploads/sites/6/2021/12/mcaxs-modelmaker-h120-en.pdf?utm_source=chatgpt.com "HANDHELD SCANNING"
[6]: https://www.crossco.com/wp-content/uploads/2022/01/Kreon-Ace-Brochure.pdf?utm_source=chatgpt.com "ACE measuring arm and Skyline scanners: - EYES - WIDE"
[7]: https://www.kreon3d.com/products/ace-measuring-arm?utm_source=chatgpt.com "Measuring arm - Ace - Inspect all industrial parts easily"
[8]: https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe/technical-specifications?utm_source=chatgpt.com "HandyPROBE Technical specifications"
[9]: https://www.creaform3d.com/en/products/coordinate-measuring-machines-handyprobe?utm_source=chatgpt.com "Coordinate measuring machines HandyProbe"
[10]: https://cdn.standards.iteh.ai/samples/63931/297d09a9fdad438490ce6f1d6fcb0db5/ISO-10360-12-2016.pdf?utm_source=chatgpt.com "INTERNATIONAL STANDARD ISO 10360-12"
[11]: https://www.iso.org/standard/54522.html?utm_source=chatgpt.com "ISO 10360-8:2013 - Geometrical product specifications ..."
[12]: https://www.kreon3d.com/article/iso-10360-12-standard-for-measuring-arms?utm_source=chatgpt.com "ISO 10360-12 standard for measuring arms - Kreon's 3D"
[13]: https://petercorke.com/books/robotics-vision-control-all-versions/?utm_source=chatgpt.com "Robotics, Vision & Control (all versions)"
[14]: https://petercorke.com/toolboxes/robotics-toolbox/?utm_source=chatgpt.com "Robotics Toolbox"
[15]: https://github.com/petercorke/robotics-toolbox-matlab?utm_source=chatgpt.com "petercorke/robotics-toolbox-matlab"
[16]: https://arxiv.org/abs/2010.08696?utm_source=chatgpt.com "A Systematic Approach to Computing the Manipulator Jacobian and Hessian using the Elementary Transform Sequence"
[17]: https://arxiv.org/abs/2207.01796?utm_source=chatgpt.com "Manipulator Differential Kinematics: Part 1: Kinematics, Velocity, and Applications"
[18]: https://www.faro.com/en/Resource-Library/Brochure/Quantum-FaroArm?utm_source=chatgpt.com "Quantum X FaroArm® Series​ | Brochure"


# Anexos